## 使用的資料說明

我在金門開了一間酒吧叫做夢酒館，這一個專案本來是學校作業，當時剛好在寫補助案，想到可以做一個 RAG 的內部資料集，因為常常在申請政府案子，很常需要重複整理有關公司資訊等等的，因此認為可以把相關資訊都全部放進 RAG，未來可以根據不同的專案，提取資料並且快速整合撰寫成不同方向的計劃書。

目前因為是作業實作，先將機器人設定為是對外的品牌介紹人員，分享給朋友體驗的時候會比較有趣。

### 目前裡面的資料有
1. 品牌介紹
2. 調酒酒單介紹
3. 相關媒體報導

### 發現比較大的問題如下:
* 複合式問句： 發現 RAG 系統一次只能處理單一問題，對於包含多個子句的複合式問句，無法精確理解並提供完整答案。
    * 目前解法：為了解決這個問題，利用 LLM 先將複合式問句拆解成多個子問題，分別進行檢索，最後再整合所有答案。

* 內容組合錯誤： 由於 RAG 系統將內容切割成 chunks，在檢索過程中，偶爾會出現內容組合錯誤，例如詢問 A 調酒卻得到 B 調酒的介紹。目前已發現問題，但仍在尋找最佳解法。
    * 目前解法：有發現問題但暫時還沒辦法完全理解和實作出解法，因此還沒有更新到程式內。

## 1. 引入必要套件與載入相關設定

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [ ]:
from openai import OpenAI
import gradio as gr
import ast
import tqdm as notebook_tqdm

In [ ]:
import os
import dotenv
dotenv.load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
model = os.getenv("OPENAI_MODEL")
embedding_model_name = os.getenv("EMBEDDING_MODEL")

print(f"api_key = {api_key}")
print(f"model = {model}")
print(f"embedding_model_name = {embedding_model_name}")

## 2. 自訂 E5 embedding 類別

In [ ]:
class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

## 3. 載入 `faiss_db`

In [ ]:
embedding_model = CustomE5Embedding(model_name=embedding_model_name)
db = FAISS.load_local("faiss_db", embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

## 4. 設定好我們要的 LLM

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

## 6. 使用 RAG 來回應

為了解決，複合式問句的檢索準確度，重新設計檢索和回答的流程


1. 語意拆解： 使用大型語言模型（LLM）將使用者輸入的複合式問題，拆解成多個語意清晰、可獨立檢索的子問題。
2. 個別檢索： 針對每個拆解後的子問題，分別從知識庫中檢索相關文件。
3. 整合回答： 根據原始問句，以及步驟二中檢索到的所有相關文件，使用 LLM 整合生成最終的完整回覆。

### 6.1 語意拆解
透過 LLM 將 user input 拆解成很多個小子句


In [ ]:
# 透過語意拆解對話成很多個小子句

system_prompt_seperate = """
你是一位專門協助語意拆解的 AI 工具設計師，任務是將使用者輸入的複合問題，轉換成清楚、可獨立檢索的子問題。

請依照以下規則進行拆解：

1. 若問題中包含兩個以上的子句（例如多個動作、主詞、或時間條件），請將其拆成多個子問題。
2. 每個子問題應該是完整的句子，能夠獨立被語意檢索模型理解。
3. 拆解後請用 Python list 格式回傳，例如：[夢酒館登過哪些國際媒體，分別是哪些調酒？]，請拆解成
["夢酒館有登過哪些國際媒體？", "夢酒館登上國際媒體的是哪些調酒？"]
4. 若問題本身已經是單一子句，請回傳原句構成的 list。
5. 除了 Python list 之外，不要回傳任何說明。

請注意：不要自行補充或改寫原始問題的語意，只做語意拆解。
"""

input_prompt_seperate = """
請將下列文字，重新拆解成不同的子句，並且整理成一個 Python list 回傳。
{queries}
"""

def seperate_queries(user_input):
    # 將自定 prompt 套入格式
    final_prompt = input_prompt_seperate.format(queries=user_input)

    # 呼叫 OpenAI API
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt_seperate},
            {"role": "user", "content": final_prompt},
        ]
    )
    raw_output = response.choices[0].message.content
    sub_queries = ast.literal_eval(raw_output)

    return {
        "main_query": user_input, # 把原始問題也保留
        "sub_queries": sub_queries
    }

# 測試與示範
user_input = "我喜歡清爽氣泡的調酒，請你推薦我一杯調酒。同時也請你跟我說明夢酒館在地方所做的貢獻，還有夢酒館登過的國際媒體報導。"

# 將主要輸入分解成多個子句
result = seperate_queries(user_input)
main_query = result['main_query']
queries_list = result['sub_queries']

# 輸出原始問句和分解後的子句
print(f"原始問句是：\n{result['main_query']}\n")
print("分解後的子問句是：")
for query in queries_list:
    i = queries_list.index(query) + 1
    print(f"子問句{i}：{query}")

### 6.2 個別檢索
分別針對每一個子句檢索資料

In [ ]:
#分別檢索取回資料並使用 top k = 3 讓資料更完整
def retrieve_answers(queries_list, top_k=3):
    results = []
    for sub_query in queries_list:
        docs = retriever.invoke(sub_query, config=dict(k=top_k)) # Gemini 說 BaseRetriever.get_relevant_documents 之後不支援，要我改成 invoke
        # 做一個空白的 list 並存入檢索後的資料
        retrieved_chunks = []

        for doc in docs:
            retrieved_chunks.append(doc.page_content)
        results.append({
            "sub_query": sub_query,
            "chunks": retrieved_chunks
        })
    return results

# 測試與示範(延續上面的 user_input)
retrieved_answers = retrieve_answers(queries_list)

# 輸出每一個子句檢索回來的答案
print(f"共有{len(retrieved_answers)}組問答，Q&A分別是:\n")

for answer_pairs in retrieved_answers:
    sub_query = answer_pairs["sub_query"]
    chunks = answer_pairs["chunks"]
    print(f"Sub_query: {sub_query}\n Chunks: {chunks}\n")

### 6.3 整合回答
把6.1 6.2得到的資料，請 LLM 整合並回答。

In [ ]:
#使用 user input　當成問句，並使用檢索之後的資料提共給　LLM 當作內容基準，可以避免分割成子句後，失去了原本完整問句的語意和情緒，但又可以透過檢索回來的資料，提升回答準確度。

system_prompt = (
    "你是『把夢酒館介紹給旅客的品牌大使』—— 一位兼具雞尾酒專業、地方文化策展力的品牌介紹人員。"
    "你的任務是依據被提共的內容，親切生動地向來訪旅客講解夢酒館的故事、雞尾酒靈感及我們在金門推動的地方創生行動。"
    "請以繁體中文作為回答語言，回答要溫暖、專業。"
    "請根據來源資料回答，若資料中未明確提及，請不要自行組合資訊。"
)

prompt_template = """
旅客問題：
{main_query}

根據下列資料來回應訪客問題，務必貼合資料細節並保持品牌語氣：
{retrieved_answers}

請依據下列規範作答：
1. 可以的話，附上相關連結或實際案例說明。
2. 如果知識庫無法支援完整答案，請誠實說明並提出可協助的後續方向 (ex. 請 Email 到 contact@mojokm.com)，可以推薦一個最接近的，但不要自行組合資訊。
3. 請一定要全程使用台灣人慣用的繁體中文來回答。
4. 在推薦調酒之前，請先了解對方的口味喜好，否則以品牌故事理念為主要回答。
5. 請避免將不同調酒的描述混合使用，除非資料中有明確比較。
"""

chat_history = []

def integrate_answers(main_query, retrieved_answers):

    # 將自定 prompt 套入格式
    final_prompt = prompt_template.format(main_query=main_query, retrieved_answers=retrieved_answers)

    # 呼叫 OpenAI API
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": final_prompt},
    ]
    )
    answer = response.choices[0].message.content

    chat_history.append((main_query, answer))
    return answer

# 測試與示範
print(f"原始問句是:\n\n「{main_query}」\n\n將會使用這句話當作主要的問句給LLM\n\n\n")

# 整合檢索之後的回答
answer = integrate_answers(main_query, retrieved_answers)
print(f"整合之後的回答是:\n\n{answer}")
print('\n\n\n')

# 7. 用 Gradio 打造 Web App

In [ ]:
# 把上面三個步驟重新整合成一個 function 給 Gradio 使用

chat_history_local = []

def respond(message, chat_history_local):
    result = seperate_queries(message)
    main_query = result['main_query']
    queries_list = result['sub_queries']
    retrieved_answers = retrieve_answers(queries_list)
    response = integrate_answers(main_query, retrieved_answers)
    chat_history_local.append({"role": "user", "content": message})  # 使用者訊息
    chat_history_local.append({"role": "assistant", "content": response})  # 機器人回應
    return "", chat_history_local

#測試與示範
res = respond(user_input, chat_history_local)
res

In [ ]:
# 清空對話紀錄
chat_history = []
chat_history_local = []

with gr.Blocks() as demo:
    gr.Markdown("""
    # 夢酒館品牌大使 \n
    您好，我是夢酒館的品牌大使，可以問我任何有關夢酒館的問題，請問有什麼我可以為您服務的嗎？\n
    可以問我有關\n
    1. 調酒推薦 2. 我們的故事 3. 其他媒體報導
    """)

    chatbot = gr.Chatbot(type="messages") # 根據 Gemini 的說明，修改成新版支援的格式。

    msg = gr.Textbox(placeholder="請輸入你對夢酒館的好奇...")

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(share=True, debug=True)

In [ ]:
# 聊完天後，輸出聊天紀錄
index = 0
for chat in chat_history:
    index +=1
    print(f"{index}:\n使用者輸入:{chat[0]}\nRAG機器人:{chat[1]}\n\n")